Importing Required Libraries

In [73]:
#Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import get_as_dataframe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Authenticate Google Sheets

In [74]:
#Authenticate Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

#Load spreadsheet by ID
sheet_id = "1LQzDOnIMUm81bLXlj6tzM_qCvUHO2ghHFMY5RW90V9k"
spreadsheet = gc.open_by_key(sheet_id)

Loading & preprocess datasets

In [75]:
#Load from Google Sheet
def load_sheet(sheet_name):
    sheet = spreadsheet.worksheet(sheet_name)
    rows = sheet.get_all_values()[3:]
    header = rows[0]
    data = rows[1:]
    return pd.DataFrame(data, columns=header)

#Load all 3 sheets
gameplay_df = load_sheet("User Gameplay data")
deposit_df = load_sheet("Deposit Data")
withdrawal_df = load_sheet("Withdrawal Data")

#Convert to numeric
gameplay_df['Games Played'] = pd.to_numeric(gameplay_df['Games Played'], errors='coerce')
deposit_df['Amount'] = pd.to_numeric(deposit_df['Amount'], errors='coerce')
withdrawal_df['Amount'] = pd.to_numeric(withdrawal_df['Amount'], errors='coerce')

#Standardize column names and fix user ID casing
for df in [gameplay_df, deposit_df, withdrawal_df]:
    df.rename(columns=lambda x: x.strip().lower(), inplace=True)
    if 'user id' not in df.columns and 'user ID' in df.columns:
        df.rename(columns={"user ID": "user id"}, inplace=True)

#Parse datetime, extract date and slot
def process_datetime_and_slot(df):
    df['datetime'] = pd.to_datetime(df['datetime'], dayfirst=True)
    df['date'] = df['datetime'].dt.date
    df['slot'] = df['datetime'].dt.hour.apply(lambda h: 'S1' if h < 12 else 'S2')
    return df

gameplay_df = process_datetime_and_slot(gameplay_df)
deposit_df = process_datetime_and_slot(deposit_df)
withdrawal_df = process_datetime_and_slot(withdrawal_df)

Loyalty Points Calculation

In [76]:
def calculate_loyalty_points(date, slot):
    g = gameplay_df[(gameplay_df['date'] == date) & (gameplay_df['slot'] == slot)]
    d = deposit_df[(deposit_df['date'] == date) & (deposit_df['slot'] == slot)]
    w = withdrawal_df[(withdrawal_df['date'] == date) & (withdrawal_df['slot'] == slot)]

    #Count deposits and withdrawals
    d_count = d.groupby('user id').size().rename("Deposit Count")
    w_count = w.groupby('user id').size().rename("Withdrawal Count")

    #Sum deposits and withdrawals
    d_sum = d.groupby('user id')['amount'].sum().rename("Deposit Amount")
    w_sum = w.groupby('user id')['amount'].sum().rename("Withdrawal Amount")

    #Games played
    g_play = g.groupby('user id')['games played'].sum().rename("Games Played")

    #Combine all into one dataframe
    df = pd.concat([d_sum, w_sum, d_count, w_count, g_play], axis=1).fillna(0)

    # Loyalty Point Formula
    df['Point_Deposit'] = 0.01 * df['Deposit Amount']
    df['Point_Withdrawal'] = 0.005 * df['Withdrawal Amount']
    df['Point_Diff'] = 0.001 * np.maximum(df['Deposit Count'] - df['Withdrawal Count'], 0)
    df['Point_Games'] = 0.2 * df['Games Played']
    df['Total Loyalty Points'] = df['Point_Deposit'] + df['Point_Withdrawal'] + df['Point_Diff'] + df['Point_Games']

    return df.reset_index()[['user id', 'Total Loyalty Points', 'Games Played']]

Solving Part A Questions

In [77]:
#PART A – Q1: Run for specific date/slot
from datetime import datetime

slots_to_check = [
    ("2023-10-02", "S1"),
    ("2023-10-16", "S2"),
    ("2023-10-18", "S1"),
    ("2023-10-26", "S2"),
]

for date_str, slot in slots_to_check:
    date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
    print(f"\n📆 {date_str} - Slot {slot}")
    result = calculate_loyalty_points(date_obj, slot)
    display(result.sort_values("Total Loyalty Points", ascending=False))


📆 2023-10-02 - Slot S1


,user id,Total Loyalty Points,Games Played



📆 2023-10-16 - Slot S2


,user id,Total Loyalty Points,Games Played



📆 2023-10-18 - Slot S1


,user id,Total Loyalty Points,Games Played



📆 2023-10-26 - Slot S2


,user id,Total Loyalty Points,Games Played


In [78]:
#PART A - Q2: Overall Loyalty Points in October & Ranking Players
def calculate_monthly_loyalty_points():
    #Count deposits and withdrawals
    d_count = deposit_df.groupby('user id').size().rename("Deposit Count")
    w_count = withdrawal_df.groupby('user id').size().rename("Withdrawal Count")

    #Sum deposits and withdrawals
    d_sum = deposit_df.groupby('user id')['amount'].sum().rename("Deposit Amount")
    w_sum = withdrawal_df.groupby('user id')['amount'].sum().rename("Withdrawal Amount")

    #Games played
    g_play = gameplay_df.groupby('user id')['games played'].sum().rename("Games Played")

    #Combine everything into one DataFrame
    df = pd.concat([d_sum, w_sum, d_count, w_count, g_play], axis=1).fillna(0)

    #Loyalty Point Calculation
    df['Point_Deposit'] = 0.01 * df['Deposit Amount']
    df['Point_Withdrawal'] = 0.005 * df['Withdrawal Amount']
    df['Point_Diff'] = 0.001 * np.maximum((df['Deposit Count'] - df['Withdrawal Count']), 0)
    df['Point_Games'] = 0.2 * df['Games Played']
    df['Total Loyalty Points'] = df['Point_Deposit'] + df['Point_Withdrawal'] + df['Point_Diff'] + df['Point_Games']

    return df.reset_index()
#Display top 10
monthly_points_df.sort_values(['Total Loyalty Points', 'Games Played'], ascending=[False, False]).head(10)

,user id,Deposit Amount,Withdrawal Amount,Deposit Count,Withdrawal Count,Games Played,Point_Deposit,Point_Withdrawal,Point_Diff,Point_Games,Total Loyalty Points,Rank


In [79]:
#PART A - Q3 What is the average deposit amount
average_deposit_amount = deposit_df['amount'].mean()
print(f"📊 Average Deposit Amount: ₹{average_deposit_amount:.2f}")

📊 Average Deposit Amount: ₹5492.19


In [80]:
#PART A - Q4 What is the average deposit amount per user in a month
# 🎯 Filter October deposits
oct_deposits = deposit_df[(deposit_df['date'].apply(lambda d: d.month == 10 and d.year == 2023))]

#Group by user and average their deposits
avg_deposit_per_user = oct_deposits.groupby('user id')['amount'].mean()

#Overall average of these user-wise averages
overall_avg_deposit_per_user = avg_deposit_per_user.mean()
print(f"📊 Average Deposit per User (October): ₹{overall_avg_deposit_per_user:.2f}")

📊 Average Deposit per User (October): ₹nan


In [81]:
#PART A - Q5 What is the average number of games played per user
# ✅ Group by user to get total games
games_per_user = gameplay_df.groupby('user id')['games played'].sum()

#Calculate average games played
average_games_played = games_per_user.mean()
print(f"🎮 Average Number of Games Played per User: {average_games_played:.2f}")

🎮 Average Number of Games Played per User: 355.27


Solving PART B Question

In [82]:
monthly_loyalty_df = calculate_monthly_loyalty_points()

#Rank players first by loyalty points, then by games played (in case of tie)
monthly_loyalty_df = monthly_loyalty_df.sort_values(
    by=["Total Loyalty Points", "Games Played"], ascending=False
).reset_index(drop=True)

#Get Top 50 Players
top_50_df = monthly_loyalty_df.head(50)

#Bonus Distribution Function
def distribute_bonus_by_loyalty(df, total_bonus=50000):
    df = df.copy()
    total_loyalty = df['Total Loyalty Points'].sum()
    df['Bonus'] = (df['Total Loyalty Points'] / total_loyalty) * total_bonus
    return df.sort_values("Bonus", ascending=False)

#Apply bonus distribution to Top 50
bonus_df = distribute_bonus_by_loyalty(top_50_df)
display(bonus_df[['user id', 'Total Loyalty Points', 'Games Played', 'Bonus']].head(10))  # Top 10 preview

,user id,Total Loyalty Points,Games Played,Bonus
0,634,83843.325,24,6638.861745
1,99,23665.737,10,1873.894625
2,672,22757.780,10,1802.000995
3,212,22199.282,1,1757.778142
4,740,19211.824,2,1521.225970
5,566,19153.755,183,1516.627965
6,714,16764.234,6,1327.421495
7,421,15446.460,1557,1223.077835
8,369,14438.444,37,1143.261357
9,30,14053.375,13,1112.770917


✅ Part C – Is the Loyalty Point Formula Fair? Suggestions for Improvement
🎯 Current Formula Recap
The loyalty point formula used:


Loyalty Points =
    0.01 × Deposit Amount +
    0.005 × Withdrawal Amount +
    0.001 × max(Deposit Count - Withdrawal Count, 0) +
    0.2 × Number of Games Played


⚖️ Fairness Evaluation
Strengths:

✅ Encourages players to play more games, which directly benefits the platform (0.2 weight).

✅ Rewards deposits and activity, which are good for monetization.

✅ Slightly discourages frequent withdrawals by rewarding net deposits.

Concerns:

❌ Heavy weight on deposits might favor players with high spending capacity, not necessarily loyalty.

❌ Withdrawal also gives points, which can be exploited by doing frequent deposits & withdrawals.

❌ Game quality, win rate, or game duration are not considered—players might spam low-effort games.

❌ No distinction between genuine gameplay and bonus abuse or transaction farming.

💡 Suggestions to Improve the Formula
1. Introduce a cap or diminishing returns on deposits
Prevents high depositors from dominating the leaderboard unfairly.

Example: Only first ₹10,000 per day gives full points.

2. Reward net deposits instead of gross deposits + withdrawals



Net Deposit = Total Deposit - Total Withdrawal
Points = 0.01 * max(Net Deposit, 0)



3. Incorporate Win Rate / Game Outcomes
Example: 0.5 extra point for each win, or normalize points based on performance.

4. Add a penalty or zero reward for suspicious repetitive deposit/withdrawal cycles
Can track user behavior patterns to flag abuse.

5. Incorporate user retention or consistency
Reward players who play regularly throughout the month, not just in bursts.

6. Different weights for different games
If multiple games exist (e.g., Ludo, Poker), assign different base points based on skill or value.